# Example notebook to load and filter a raw data in S3

In [ ]:
import sys
import os

In [ ]:
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "...")))
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import core.daphmeIO as loader
import core.filters as filters

We load a sample of Gravy trajectory data for the Philadelphia area

In [ ]:
part_path = '../data/sample3/date=2024-01-07/aad4a23f7a90441aa0f55f06e5e4313d-0.parquet'
part_path = "s3://phl-pings/gravy_clean/date=2019-11-01/part-00007-a7eb387d-1b0c-4aa7-b6a1-47023f1940bd.c000.snappy.parquet"

traj_cols =  {"user_id":"identifier",
              "x":"x",
              "y":"y",
              "datetime":"local_timestamp",
              "timestamp":"timestamp"}

### Get a sample of users

In [ ]:
u_sample = loader.sample_users(part_path, format='parquet', frac_users=0.2, user_id='identifier')

### Load data for users in u_sample for 3 days

In [ ]:
filepath = ['s3://phl-pings/gravy_clean/date=2019-11-01/',
            's3://phl-pings/gravy_clean/date=2019-11-02/',
            's3://phl-pings/gravy_clean/date=2019-11-03/',
            's3://phl-pings/gravy_clean/date=2019-11-04/',
            's3://phl-pings/gravy_clean/date=2019-11-05/',
            's3://phl-pings/gravy_clean/date=2019-11-06/']

data = loader.sample_from_file(part_path, users=u_sample, format='parquet', traj_cols=traj_cols, user_id = 'identifier')

In [ ]:
data['timestamp'] = data[traj_cols['datetime']].astype(int) // 10**9

### Project coordinates to Web Mercator

In [ ]:
data = filters.to_projection(data, x='x', y='y')

### Compute the q-statistic for the users in this sample

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, Point

from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, DoubleType


# user can pass latitude and longitude as kwargs, user can pass x and y, OR traj_cols (prioritizing latitude, longitude). 
def to_projection(df: pd.DataFrame,
                  latitude: str,
                  longitude: str,
                  from_crs: str = "EPSG:4326",
                  to_crs: str = "EPSG:3857",
                  spark_session: SparkSession = None):
    """
    Projects latitude and longitude columns from one CRS to another.

    Parameters
    ----------
    df : pd.DataFrame
        Input DataFrame containing latitude and longitude columns.
    latitude : str
        Name of the latitude column.
    longitude : str
        Name of the longitude column.
    from_crs : str, optional
        EPSG code for the original CRS (default is "EPSG:4326").
    to_crs : str, optional
        EPSG code for the target CRS (default is "EPSG:3857").

    Returns
    -------
    pd.DataFrame
        DataFrame with new 'x' and 'y' columns representing projected coordinates.
    """
    if spark_session:
        pass

    else:
        if latitude not in df.columns or longitude not in df.columns:
            raise ValueError(f"Latitude or longitude columns '{latitude}', '{longitude}' not found in DataFrame.")

        proj_cols = _to_projection(df[latitude],
                                   df[longitude],
                                   from_crs,
                                   to_crs)

        df['x'] = proj_cols['x']
        df['y'] = proj_cols['y']

    return df


def _to_projection(lat_col,
                   long_col,
                   from_crs: str,
                   to_crs: str):
    """
    Helper function to project latitude/longitude columns to a new CRS.
    """
    gdf = gpd.GeoSeries(gpd.points_from_xy(long_col, lat_col),
                        crs=from_crs)
    projected = gdf.to_crs(to_crs)
    output = pd.DataFrame({'x': projected.x, 'y': projected.y})

    return output


def filter_to_box(df: pd.DataFrame,
                  latitude: str,
                  longitude: str,
                  polygon: Polygon,
                  spark_session: SparkSession = None):
    '''
    Filters DataFrame to keep points within a specified polygon's bounds.

    Parameters
    ----------
    df : pd.DataFrame
        Input DataFrame with latitude and longitude columns.
    polygon : shapely.geometry.Polygon
        Polygon defining the area to retain points within.
    latitude : str
        Name of the latitude column.
    longitude : str
        Name of the longitude column.

    Returns
    -------
    pd.DataFrame
        Filtered DataFrame with points inside the polygon's bounds.
    '''
    if spark_session:
        pass

    else:
        if not isinstance(polygon, Polygon):
            raise TypeError("Polygon parameter must be a Shapely Polygon object.")

        if latitude not in df.columns or longitude not in df.columns:
            raise ValueError(f"Latitude or longitude columns '{latitude}', '{longitude}' not found in DataFrame.")

        min_x, min_y, max_x, max_y = polygon.bounds

        # TO DO: handle different column names and/or defaults as in daphmeIO. i.e. traj_cols as parameter

        return df[(df[longitude].between(min_y, max_y)) & (df[latitude].between(min_x, max_x))]


def _filter_to_box_spark(df: pd.DataFrame,
                         bounding_wkt: str,
                         spark: SparkSession,
                         longitude_col: str,
                         latitude_col: str,
                         id_col: str):
    """Filters a DataFrame based on whether geographical points
    (defined by longitude and latitude) fall within a specified geometry.

    Parameters
    ----------
    df : DataFrame
        The Spark DataFrame to be filtered. It should contain columns
        corresponding to longitude and latitude values, as well as an id column.

    bounding_wkt : str
        The Well-Known Text (WKT) string representing the bounding geometry
        within which points are tested for inclusion. The WKT should define
        a polygon in the EPSG:4326 coordinate reference system.

    spark : SparkSession
        The active SparkSession instance used to execute Spark operations.

    longitude_col : str, default "longitude"
        The name of the column in 'df' containing longitude values. Longitude
        values should be in the EPSG:4326 coordinate reference system.

    latitude_col : str, default "latitude"
        The name of the column in 'df' containing latitude values. Latitude
        values should be in the EPSG:4326 coordinate reference system.

    id_col : str, default "id"
        The name of the column in 'df' containing user IDs.

    Returns
    ----------
    DataFrame
        A new Spark DataFrame filtered to include only rows where the point
        (longitude, latitude) falls within the specified geometric boundary
        defined by 'bounding_wkt'. This DataFrame includes all original columns
        from 'df' and an additional column 'in_geo' that is true if the point
        falls within the specified geometric boundary and false otherwise.
    """

    df = df.withColumn("coordinate", F.expr(f"ST_MakePoint({longitude_col}, {latitude_col})"))
    df.createOrReplaceTempView("temp_df")

    query = f"""
        WITH temp_df AS (
            SELECT *,
                   ST_Contains(ST_GeomFromWKT('{bounding_wkt}'), coordinate) AS in_geo
            FROM temp_df
        ),

        UniqueIDs AS (
            SELECT DISTINCT {id_col} 
            FROM temp_df
            WHERE in_geo
        )

        SELECT t.*
        FROM temp_df t
        WHERE t.{id_col} IN (SELECT {id_col} FROM UniqueIDs)
        """

    return spark.sql(query)


def coarse_filter(df: pd.DataFrame):
    pass


def _filtered_users(df: pd.DataFrame,
                    k: int,
                    T0: str,
                    T1: str,
                    polygon: Polygon,
                    user_col: str,
                    timestamp_col: str,
                    latitude_col: str,
                    longitude_col: str) -> pd.DataFrame:
    """
    Subsets to users who have at least k distinct days with pings in the polygon 
    within the timeframe T0 to T1.

    Parameters
    ----------
    df : pd.DataFrame
        The input DataFrame containing user data with latitude, longitude, and timestamp.
    k : int
        Minimum number of distinct days with pings inside the polygon for the user to be retained.
    T0 : str
        Start of the timeframe (as a string, or datetime).
    T1 : str
        End of the timeframe (as a string, or datetime).
    polygon : Polygon
        The polygon to check whether pings are inside.
    user_col : str
        Name of the column containing user identifiers.
    timestamp_col : str
        Name of the column containing timestamps (as strings or datetime).
    latitude : str
        Name of the column containing latitude values.
    longitude : str
        Name of the column containing longitude values.

    Returns
    -------
    pd.Series
        A Series containing the user IDs for users who have at 
        least k distinct days with pings inside the polygon.
    """
    df[timestamp_col] = pd.to_datetime(df[timestamp_col])
    df_filtered = df[(df[timestamp_col] >= T0) & (df[timestamp_col] <= T1)]
    df_filtered = _in_geo(df_filtered, latitude_col, longitude_col, polygon)
    df_filtered['date'] = df_filtered[timestamp_col].dt.date

    filtered_users = (
        df_filtered[df_filtered['in_geo'] == 1]
        .groupby(user_col)['date']
        .nunique()
        .reset_index()
    )

    filtered_users = filtered_users[filtered_users['date'] >= k][user_col]

    return filtered_users


def _in_geo(df: pd.DataFrame,
            latitude_col: str,
            longitude_col: str,
            polygon: Polygon) -> pd.DataFrame:
    """
    Adds a new column to the DataFrame indicating whether points are 
    inside the polygon (1) or not (0).
    """

    def _point_in_polygon(lat, lon):
        point = Point(lat, lon)
        return 1 if polygon.contains(point) else 0

    df['in_geo'] = df.apply(lambda row: _point_in_polygon(row[latitude_col], row[longitude_col]), axis=1)

    return df


def q_filter(df: pd.DataFrame,
             qbar: float,
             user_col: str,
             timestamp_col: str):
    """
    Computes the q statistic for each user as the proportion of unique hours with pings 
    over the total observed hours (last hour - first hour) and filters users where q > qbar.

    Parameters
    ----------
    df : pd.DataFrame
        A DataFrame containing user IDs and timestamps.
    user_col : str
        The name of the column containing user IDs.
    timestamp_col : str
        The name of the column containing timestamps.
    qbar : float
        The threshold q value; users with q > qbar will be retained.

    Returns
    -------
    pd.Series
        A Series containing the user IDs for users whose q_stat > qbar.
    """
    df[timestamp_col] = pd.to_datetime(df[timestamp_col])

    user_q_stats = df.groupby(user_col).apply(
        lambda group: _compute_q_stat(group, timestamp_col)
    ).reset_index(name='q_stat')

    # Filter users where q > qbar
    filtered_users = user_q_stats[user_q_stats['q_stat'] > qbar][user_col]

    return filtered_users

def q_stats(df: pd.DataFrame, user_id: str, timestamp: str):
    """
    Computes the q statistic for each user as the proportion of unique hours with pings 
    over the total observed hours (last hour - first hour).

    Parameters
    ----------
    df : pd.DataFrame
        A DataFrame containing user IDs and timestamps.
    user_id : str
        The name of the column containing user IDs.
    timestamp_col : str
        The name of the column containing timestamps.

    Returns
    -------
    pd.DataFrame
        A DataFrame containing each user and their respective q_stat.
    """
    datetime = pd.to_datetime(df[timestamp])

    q_stats = df.groupby(user_id).apply(
        lambda group: _compute_q_stat(group, datetime)
    ).reset_index(name='q_stat')

    return q_stats


def _compute_q_stat(user, datetime):
    date_hour = datetime.dt.to_period('h')
    unique_hours = date_hour.nunique()

    # Calculate total observed hours (difference between last and first hour)
    first_hour = date_hour.min()
    last_hour = date_hour.max()
    # maybe it should be + 1 hour
    total_hours = (last_hour - first_hour).total_seconds() / 3600

    # Compute q as the proportion of unique hours to total hours
    q_stat = unique_hours / total_hours if total_hours > 0 else 0
    if q_stat>1:
        pdb.set_trace()
    return q_stat

In [ ]:
q = q_stats(data, user_id= traj_cols['user_id'], timestamp= traj_cols['timestamp'])

In [ ]:
# Plotting the histogram

plt.figure(figsize=(4, 3))
plt.hist(q_stats['q_stat'], bins=20, edgecolor='black')
plt.xlabel('Q Statistic')
plt.ylabel('Frequency')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()

plt.show()